```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Implicit Equal Weights Particle Filter

This notebook implements prototyping and example/demo of the Implicit Equal Weights Particle Filter (IEWPF).


## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from scipy.special import lambertw

import pyopencl
import os
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')
matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

#Import our simulator
from SWESimulators import CDKLM16, PlotHelper, Common

from SWESimulators import BathymetryAndICs as BC
from SWESimulators import OceanStateNoise
from SWESimulators import OceanNoiseEnsemble
from SWESimulators import BaseOceanStateEnsemble
from SWESimulators import DataAssimilationUtils as dautils


In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
cl_queue = pyopencl.CommandQueue(cl_ctx)
print "Using ", cl_ctx.devices[0].name

# Ensemble

We need an ensemble where each particle
- runs an independent ocean model
- applies a localized small-scale error
- observes the drifter prosition of the syntetic truth, and estimate the underlying velocity field at that point
- look-up the particles' velocity at the same point as where the drifter was observed

Needs to be done:
- Initialize models (create netcdf with init, add error with amp 10*q0(?), put drifter into a small area of the 
- make useful plots to evaluate the results
    - Suggestion: 3-line [eta, hu, hv] plot, with truth, ensemble (mean field with individual drifters), mean-square diff?
    - 3x3/4x4/5x5 plot of eta from different ensemble members?
    - Standard animation of a single ensemble member.


## Create initial condition for ensemble:

In [ ]:
# DEFINE PARAMETERS

#Coriolis well balanced reconstruction scheme
nx = 40
ny = 40

dx = 4.0
dy = 4.0

dt = 0.05
g = 9.81
r = 0.0

f = 0.05
beta = 0.0

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

# Define which cell index which has lower left corner as position (0,0)
x_zero_ref = 2
y_zero_ref = 2

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
dataShapeHi = (ny + ghosts[0]+ghosts[2]+1, 
             nx + ghosts[1]+ghosts[3]+1)

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
eta0_extra = np.zeros(dataShape, dtype=np.float32, order='C')
hv0 = np.zeros(dataShape, dtype=np.float32, order='C');
hu0 = np.zeros(dataShape, dtype=np.float32, order='C');
waterDepth = 10.0
Hi = np.ones(dataShapeHi, dtype=np.float32, order='C')*waterDepth

# Add disturbance:
if True:
    rel_grid_size = nx*1.0/dx
    BC.addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05*rel_grid_size, validDomain)
    eta0 = eta0*0.3
    BC.addBump(eta0, nx, ny, dx, dy, 0.7, 0.3, 0.10*rel_grid_size, validDomain)
    eta0 = eta0*(-1.3)
    BC.addBump(eta0, nx, ny, dx, dy, 0.15, 0.8, 0.03*rel_grid_size, validDomain)
    eta0 = eta0*1.0
    BC.addBump(eta0, nx, ny, dx, dy, 0.6, 0.75, 0.06*rel_grid_size, validDomain)
    BC.addBump(eta0, nx, ny, dx, dy, 0.2, 0.2, 0.01*rel_grid_size, validDomain)
    eta0 = eta0*(-0.03)
    BC.addBump(eta0_extra, nx, ny, dx, dy, 0.5, 0.5, 0.4*rel_grid_size, validDomain)
    eta0 = eta0 + 0.02*eta0_extra
    BC.initializeBalancedVelocityField(eta0, Hi, hu0, hv0, f, beta, g, nx, ny, dx ,dy, ghosts)
    eta0 = eta0*0.5


if 'sim' in globals():
    sim.cleanUp()
if 'ensemble' in globals():
    ensemble.cleanUp()
    
q0 = 0.5*dt*f/(g*waterDepth)
print "q0: ", q0
print "[f, g, H]", [f, g, waterDepth]
print "f/gH: ", f/(g*waterDepth)
print "gH/f: ", g*waterDepth/f

reload(CDKLM16)
reload(BaseOceanStateEnsemble)
reload(OceanNoiseEnsemble)
reload(PlotHelper)
sim = CDKLM16.CDKLM16(cl_ctx, eta0, hu0, hv0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions, \
                      write_netcdf=False, \
                      small_scale_perturbation=True, \
                      small_scale_perturbation_amplitude=q0)

ensemble_size = 3

ensemble = OceanNoiseEnsemble.OceanNoiseEnsemble(ensemble_size, cl_ctx,  
                                                 observation_type=dautils.ObservationType.DirectUnderlyingFlow)
ensemble.setGridInfoFromSim(sim)
ensemble.setStochasticVariables(#observation_variance_factor=2.0,
                                observation_variance = 0.01**2,
                                small_scale_perturbation_amplitude=q0)
                                #initialization_variance_factor_ocean_field=50)
ensemble.init()

fig = plt.figure()
plotter = PlotHelper.EnsembleAnimator(fig, ensemble, trueStateOnly=True)

T = 2
sub_t = 300*dt
def animate(i):
    if (i>0):
        t = ensemble.step(sub_t)
    else:
        t = 0.0

    plotter.plot(ensemble);

    fig.suptitle("Ensemble = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) 

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
ensemble.plotEnsemble()

In [ ]:
max_dt = ensemble.findLargestPossibleTimeStep()
print "Largest possible timestep with this case: ", max_dt

In [ ]:
fig = ensemble.plotDistanceInfo()

# Implementing IEWPF

The IEWPF algorithm for the observation at time $t^m$ is applyed to the model at time $t^{m-1}$.
The algorithm consists of the following steps:
1. Find target weight using only deterministic evolution of the model from $t^{m-1}$ to $t^m$.
- Draw a sample from $\xi \sim N(0,P)$.
- Move particles towards observation using a applying some scaling of $\xi$

Some challenges:
- As long as we have one and only one drifter, constant $H(x,y)$ and double periodic boundary conditions, the linear problem within step 1 has the same matrix for any observation. This matrix should therefore be pre-calculated.
- During the creation of $\xi$ we need the random numbers, so using functionality of the OceanNoiseClass would be nice.


Questions:
- Should the IEWPF subroutines/functions/methods be under its own class? Or under the OceanNoiseClass

## Covariance structures
The model error is sampled from $N(0, Q)$, where 
$$Q = Q^{1/2} Q^{1/2,T} = U_{GB} \tilde{Q}^{1/2} \tilde{Q}^{1/2} U_{GB}^T.$$
Here, $\tilde{Q}^{1/2}$ is the SOAR function, and $U_{GB}$ is the operator that map $\eta$ to $[\eta, hu, hv]^T$, such that the state is in geostrophic balance.

## Step 0
Rum the model to observation time, obtain w_rest, and run the particles to observation time without using the stochastic term.

In [ ]:
t = ensemble.step_truth(dt, stochastic=True)
#w_rest = -np.log(ensemble.getGaussianWeight())
w_rest = -np.log(np.ones(ensemble.getNumParticles())/(1.0 + ensemble.getNumParticles()))
t = ensemble.step_particles(dt, stochastic=False)

## Step 1

The full covariance matrix can be written as
$$ Q = Q^{1/2} Q^{1/2,T} = U_{GB} \tilde{Q}^{1/2} \tilde{Q}^{1/2} U_{GB}^T $$
so that 
$$ HQH^T = H U_{GB} \tilde{Q}^{1/2} \tilde{Q}^{1/2} U_{GB}^T  H^T$$

For Step 1 we first need to find $$S := (H Q H^T + R)^-1,$$ which we also will need for later.
We therefore store S, before finding the target weight.

In [ ]:
debug = False

def showMatrices(x, y, title, z = None):
    num_cols = 2
    if z is not None:
        num_cols = 3
    fig = plt.figure(figsize=(num_cols*2,2))
    plt.subplot(1,num_cols,1)
    plt.imshow(x.copy(), origin="lower", interpolation="None")
    plt.xlabel('(%.2E, %.2E)' % (np.min(x), np.max(x)))
    plt.subplot(1,num_cols,2)
    plt.imshow(y.copy(), origin="lower", interpolation="None")
    plt.xlabel('(%.2E, %.2E)' % (np.min(y), np.max(y)))
    if z is not None:
        plt.subplot(1, num_cols, 3)
        plt.imshow(z.copy(), origin="lower", interpolation="None")
        plt.xlabel('(%.2E, %.2E)' % (np.min(z), np.max(z)))
    plt.suptitle(title)
    
# Copied from RandomNumberGenerator.ipynb  
def SOAR_Q(a_x, a_y, b_x, b_y, dx, dy, q0, L):
    dist = np.sqrt( dx*dx*(a_x - b_x)**2  +  dy*dy*(a_y - b_y)**2)
    return q0*(1.0 + dist/L)*np.exp(-dist/L)

debug = True
def createS(ensemble, const_H, debug=False):
    """
    Create the 2x2 matrix S = (HQH^T + R)^-1
    
    Constant as long as
     - one drifter only,
     - H(x,y) = const, and
     - double periodic boundary conditions
    """
    
    dt = ensemble.dt
    dx = ensemble.dx
    dy = ensemble.dy
    geoBalanceConst = ensemble.g*const_H/(2.0*ensemble.f)
    
    # These should be read from a OceanStateNoise object?
    q0 = ensemble.small_scale_perturbation_amplitude
    L = 0.75*dx 
    
    # Local storage for x and y correlations:
    x_corr = np.zeros((7,7))
    y_corr = np.zeros((7,7))
    tmp_x = np.zeros((7,7))
    tmp_y = np.zeros((7,7))
    
    # Mid_coordinates:
    mid_i, mid_j = 3, 3
    
    # Fill the buffers with U_{GB}^T H^T
    x_corr[mid_j+1, mid_i] = -geoBalanceConst/dy
    x_corr[mid_j-1, mid_i] =  geoBalanceConst/dy
    y_corr[mid_j, mid_i+1] =  geoBalanceConst/dx
    y_corr[mid_j, mid_i-1] = -geoBalanceConst/dx
    if debug: showMatrices(x_corr, y_corr, "$U_{GB}^T  H^T$")
    
    # Apply the SOAR function to fill x and y with 7x5 and 5x7 respectively
    # First for x:
    for j,i in [mid_j+1, mid_i], [mid_j-1, mid_i]:
        for b in range(j-2, j+3):
            for a in range(i-2, i+3):
                tmp_x[b, a] += x_corr[j,i]*SOAR_Q(a, b, i, j, dx, dy, q0, L)
    # Then for y:
    for j,i in [mid_j, mid_i+1], [mid_j, mid_i-1]:
        for b in range(j-2, j+3):
            for a in range(i-2, i+3):
                #print SOAR_Q(a, b, i, j, dx, dy, q0, L)
                tmp_y[b, a] += y_corr[j,i]*SOAR_Q(a, b, i, j, dx, dy, q0, L)
    if debug: showMatrices(tmp_x, tmp_y, "$Q_{SOAR} U_{GB}^T H^T$")   
        
    # Apply the SOARfunction again to fill the points needed to find drift in (mid_i, mid_j)
    # For both x and y:
    # This means that we only need to evaluate Q_{SOAR} Q_{SOAR} U_{GB}^T H^T at four points
    for j,i in [mid_j+1, mid_i], [mid_j-1, mid_i], [mid_j, mid_i-1], [mid_j, mid_i+1]:
        x_corr[j,i] = 0
        y_corr[j,i] = 0
        for b in range(j-2, j+3):
            for a in range(i-2, i+3):
                SOAR_Q_res = SOAR_Q(a, b, i, j, dx, dy, q0, L)
                x_corr[j,i] += tmp_x[b, a]*SOAR_Q_res
                y_corr[j,i] += tmp_y[b, a]*SOAR_Q_res
        if debug: print "(j, i ,x_corr[j,i], y_corr[j,i]): ", (j, i ,x_corr[j,i], y_corr[j,i])
    if debug: showMatrices(x_corr, y_corr, "$Q_{SOAR} Q_{SOAR} U_{GB}^T H^T$")
       
    # geostrophic balance:
    x_hu = -geoBalanceConst*(x_corr[mid_j+1, mid_i  ] - x_corr[mid_j-1, mid_i  ])/dy
    x_hv =  geoBalanceConst*(x_corr[mid_j  , mid_i+1] - x_corr[mid_j  , mid_i-1])/dx
    y_hu = -geoBalanceConst*(y_corr[mid_j+1, mid_i  ] - y_corr[mid_j-1, mid_i  ])/dy
    y_hv =  geoBalanceConst*(y_corr[mid_j  , mid_i+1] - y_corr[mid_j  , mid_i-1])/dx 
    
    # Structure the information as a  
    HQHT = np.matrix([[x_hu, y_hu],[x_hv, y_hv]])    
    if debug: print "HQHT\n", HQHT
    if debug: print "ensemble.observation_cov\n", ensemble.observation_cov
    S_inv = HQHT + ensemble.observation_cov
    if debug: print "S_inv\n", S_inv
    S = np.linalg.inv(S_inv)
    if debug: print "S\n", S
    return S

print "-----------"
S = createS(ensemble, 10.0, debug=True)
print "-----------"
print "S: ", S

print "ensemble.observation_cov"
print ensemble.observation_cov
print "np.linalg.inv(ensemble.observation_cov)"
print np.linalg.inv(ensemble.observation_cov)



We expected $HQH^T$ to be a full $2 \times 2$ matrix, but we see now that the $x$ and $y$ coordinate of the drifter is uncorrelated.

When one thinks about it, this makes sense. Even though we know the $x$ position of a drifter, we would have no knowledge of the $y$ position of it. 

The same argument can be used about the velocities. Can we really say anything about the $hu$ in a cell based on the value of $hv$ in the same cell? From $hu(x_i, y_j)$ alone, we can not say whether all the momentum is in $x$-direction, or if it is just a part of the total momentum, which also has a $y$ component. Even though the geostrophic balance generate rotating velocity fields, we will have no knowledge on where in the rotation we are. 

In [ ]:
# As we have S = (HQH^T + R)^-1, we can do step 1 of the IEWPF algorithm
def obtainTargetWeight(ensemble, S, w_rest, debug=False):
    d = ensemble.getInnovations()
    Ne = ensemble.getNumParticles()
    c = np.zeros(Ne)
    for i in range(Ne):
        e = np.dot(S, d[i,:])
        c[i] = w_rest[i] + 0.5*np.dot(e,d[i,:])
        #c[i] =  0.5*np.dot(e,d[i,:])
        if debug: print "c[" + str(i) + "]: ", c[i]
        if debug: print "exp(-c[" + str(i) + "]: ", np.exp(-c[i])
    return np.min(c)

target_weight = obtainTargetWeight(ensemble, S, w_rest, True)
print "target_weight: ", target_weight
print "current weights: "
print ensemble.getGaussianWeight()

## Step 2
Draw samples from $\xi \sim N(0, P)$, where 
$$ P = (Q^{-1}+ H^T R^{-1} H)^{-1}. $$
With some additional linear algebra magic, $P$ can also be written as
$$ P = Q - QH^T (HQH^T + R)^{-1} H Q.$$
Defining $S := (HQH^T + R)^{-1}$, it is straight forward to write $P$ as
$$P = Q - Q H^T S H Q \\ = Q^{1/2}(I - Q^{1/2, T}H^T S H Q^{1/2}) Q^{1/2, T}$$
By using the two-step covariance structure, we get
$$ P = U_{GB} \tilde{Q}^{1/2} \left[ I - \tilde{Q}^{1/2} U_{GB}^T H^T S H  U_{GB} \tilde{Q}^{1/2} \right] \tilde{Q}^{1/2} U_{GB}^T $$

In order to produce $\xi \sim N(0,P)$, we draw $\tilde{\xi} \sim N(0, I)$, where $\tilde{\xi} \in \mathbb{R}^{N_x}$, and find $\xi = P^{1/2} \tilde{\xi}$.
We can write
$$P = P^{1/2} P^{1/2, T},$$
if we can find the singular value decomposition (SVD) of the paranthesis in the expression for $P$.

Assume that we can write
$$ U \Sigma V^H = I - \tilde{Q}^{1/2} U_{GB}^T H^T S H  U_{GB} \tilde{Q}^{1/2},$$
we can write 
$$ P = U_{GB} \tilde{Q}^{1/2} U \Sigma^{1/2} \Sigma^{1/2} V^H \tilde{Q}^{1/2} U_{GB}^T.$$

We can now draw $\xi \sim N(0,P)$ as
$$ \xi = U_{GB} \tilde{Q}^{1/2} U \Sigma^{1/2} \tilde{\xi}$$.



### Note

Since $H$ depends on the position of the drifter, the SVD also depends on the position of the drifter. However, as long as 
- the depth is constant over the entire domain,
- $\Delta x = \Delta y$,
- $f$ is constant, and 
- there are periodic boundary conditions,

the covariance structure around the observed drifter is the same and independent of the drifters position.
We can then create this structure only once, and apply it to different parts of $\tilde{\xi}$.

The non-identity part of the SVD input is an area consisting of $7 \times 7$ cells, meaning that $U \Sigma^{1/2}$ will be a $49 \times 49$ matrix.




In [ ]:
def _periodic_SOAR_Q(a_x, a_y, b_x, b_y, dx, dy, nx, ny, q0, L):
    dist_x = min((a_x - b_x)**2, (a_x - (b_x + nx))**2, (a_x - (b_x - nx))**2)
    dist_y = min((a_y - b_y)**2, (a_y - (b_y + ny))**2, (a_y - (b_y - ny))**2)
    
    dist = np.sqrt( dx*dx*dist_x  +  dy*dy*dist_y)
    
    return q0*(1.0 + dist/L)*np.exp(-dist/L)

def _createCutoffSOARMatrixQ(ensemble, nx=None, ny=None, cutoff=2):
    if nx is None:
        nx = ensemble.nx
    if ny is None:
        ny = ensemble.ny
    dx = ensemble.dx
    dy = ensemble.dy
    q0 = ensemble.small_scale_perturbation_amplitude
    
    # Hard-coded elsewhere:
    L = 0.75*dx
    
    Q = np.zeros((ny*nx, ny*nx))
    for a_y in range(ny):
        for a_x in range(nx):
            j = a_y*nx + a_x
            for b_y in range(a_y-cutoff, a_y+cutoff+1):
                if b_y < 0:    
                     b_y = b_y + ny
                if b_y > ny-1: 
                    b_y = b_y - ny
                for b_x in range(a_x-cutoff, a_x+cutoff+1):
                    if b_x < 0:
                        b_x = b_x + nx
                    if b_x > nx-1: 
                        b_x = b_x - nx
                    i = b_y*nx + b_x
                    Q[j, i] = _periodic_SOAR_Q(a_x, a_y, b_x, b_y, dx, dy, nx, ny, q0, L)
    return Q


def _createUGBmatrix(ensemble, nx=None, ny=None):
    if nx is None:
        nx = ensemble.nx
    if ny is None:
        ny = ensemble.ny
    dx = ensemble.dx
    dy = ensemble.dy
    g  = ensemble.g
    H  = np.max(ensemble.base_H)
    f  = ensemble.f
    
    
    newCorrect = True
    print "newCorrect: ", newCorrect
    
    I = np.eye(nx*ny)
    A_hu = np.zeros((ny*nx, ny*nx))
    A_hv = np.zeros((ny*nx, ny*nx))
    for a_y in range(ny):
        for a_x in range(nx):
            if newCorrect:
                j = a_y*nx + a_x
                
                # geo balance for hu:
                i = (a_y+1)*nx + a_x
                if a_y == ny-1:
                    i = 0*nx + a_x
                A_hu[j,i] = 1.0
                i = (a_y-1)*nx + a_x
                if a_y == 0:
                    i = (ny-1)*nx + a_x
                A_hu[j,i] = -1.0

                # geo balance for hv:
                i = a_y*nx + a_x + 1
                if a_x == nx-1:
                    i = a_y*nx + 0
                A_hv[j,i] = 1.0

                i = a_y*nx + a_x - 1
                if a_x == 0:
                    i = a_y*nx + nx - 1
                A_hv[j,i] = -1.0
            
            else:
                i = a_y*nx + a_x

                # geo balance for hu:
                j = (a_y+1)*nx + a_x
                if a_y == ny-1:
                    j = 0*nx + a_x
                A_hu[j,i] = 1.0
                j = (a_y-1)*nx + a_x
                if a_y == 0:
                    j = (ny-1)*nx + a_x
                A_hu[j,i] = -1.0

                # geo balance for hv:
                j = a_y*nx + a_x + 1
                if a_x == nx-1:
                    j = a_y*nx + 0
                A_hv[j,i] = 1.0

                j = a_y*nx + a_x - 1
                if a_x == 0:
                    j = a_y*nx + nx - 1
                A_hv[j,i] = -1.0
            
    A_hu *= -g*H/(f*2*dy)
    A_hv *=  g*H/(f*2*dx)
            
    return np.bmat([[I], [A_hu], [A_hv]])

def _createMatrixH(nx, ny, pos_x, pos_y):
    H = np.zeros((2, 3*nx*ny))
    index = pos_y*nx + pos_x
    H[0, 1*nx*ny + index] = 1
    H[1, 2*nx*ny + index] = 1
    return H

def generateLocaleSVDforP(ensemble, S, debug=False):
    """
    Generates the local square root of the SVD-block needed for P^1/2.
    
    Finding:   U*Sigma*V^H = I - Q*U_GB^T*H^T*S*H*U_GB*Q
    Returning: U*sqrt(Sigma)
    """
    
    # Since the structure of the SVD-block is the same for any drifter position, we build the block
    # on a 7x7 domain with the observation in the middle cell
    local_nx = 7
    local_ny = 7
    pos_x = 3
    pos_y = 3
    
    # Create the matrices needed
    H      = _createMatrixH(local_nx, local_ny, pos_x, pos_y)
    Q_soar = _createCutoffSOARMatrixQ(ensemble, nx=local_nx, ny=local_ny)
    U_GB   = _createUGBmatrix(ensemble, nx=local_nx, ny=local_ny)
    
    UQ = np.dot(U_GB, Q_soar)
    HUQ = np.dot(H, UQ)
    SHUQ = np.dot(S, HUQ)
    HTSHUQ = np.dot(H.transpose(), SHUQ)
    UTHTSHUQ = np.dot(U_GB.transpose(), HTSHUQ)
    QUTHTSHUQ = np.dot(Q_soar, UTHTSHUQ)
    
    svd_input = np.eye(local_nx*local_nx) - QUTHTSHUQ
    
    u, s, vh = np.linalg.svd(svd_input, full_matrices=True)
    
    if debug:
        SVD_prod = np.dot(u, np.dot(np.diag(s), vh))
        fig = plt.figure(figsize=(4,4))
        plt.imshow(SVD_prod, interpolation="None")
        plt.title("SVD_prod")
        plt.colorbar()
        
        fig = plt.figure(figsize=(4,4))
        plt.imshow(SVD_prod - np.eye(49), interpolation="None")
        plt.title("SVD_prod - I")
        plt.colorbar()
        
        fig = plt.figure(figsize=(4,4))
        plt.imshow(u, interpolation="None")
        plt.title("u")
        plt.colorbar()
        
        
    return np.dot(u, np.diag(np.sqrt(s)))
    
debug = False
local_sqrt_term = generateLocaleSVDforP(ensemble, S, debug=True)
print "local_sqrt_term.shape: ", local_sqrt_term.shape
if debug:
    fig = plt.figure(figsize=(4,4))
    plt.imshow(local_sqrt_term, interpolation="None")
    plt.title("local_sqrt_term")
    plt.colorbar()

In [ ]:
if 'noise' in globals():
    noise.cleanUp()
    
#noise = OceanStateNoise.OceanStateNoise.fromsim(ensemble.particles[0], 
#                                                soar_q0=ensemble.small_scale_perturbation_amplitude)

# This is the function that needs to be implemented on GPU
def _apply_local_SVD_to_global_xi(local_sqrt_term, global_xi, nx, ny, pos_x, pos_y):
    """
    Despite the bad name, this is a good function!
    
    It takes as input:
     - local sqrt(SVD) as U*sqrt(Sigma) in a (49, 49) buffer 
     - the global xi stored in a (ny, nx) buffer
     
    The second buffer is modified so that xi = U*sqrt(Sigma)*xi
    
    Note that we have to make a copy of xi so that we don't read already updated values.
    """
    
    
    # Copy the result (representing the multiplication with I)
    read_global_xi = global_xi.copy()
    
    # Read the non-zero structure from tildeP to tildeP_block
    for loc_y_j in range(7):
        global_y_j = pos_y - 3 + loc_y_j
        for loc_x_j in range(7):
            global_x_j = pos_x - 3 + loc_x_j
            
            global_j = global_y_j*nx + global_x_j
            local_j = loc_y_j*7 + loc_x_j
            
            #loc_vec[local_j] = glob_vec[global_j]
            
            xi_j = 0.0
            for loc_y_i in range(7):
                global_y_i = pos_y - 3 + loc_y_i
                for loc_x_i in range(7):
                    global_x_i = pos_x - 3 + loc_x_i
                    
                    global_i = global_y_i*nx + global_x_i
                    local_i = loc_y_i*7 + loc_x_i
                    
                    xi_j += local_sqrt_term[local_j, local_i]*read_global_xi[global_y_i, global_x_i]
            
            global_xi[global_y_j, global_x_j] = xi_j


def drawFromP(S, local_SVD_sqrt, sim, drifter_pos, const_H, debug=False):
    nx, ny = sim.nx, sim.ny
    
    # 0) Find the cell index for the noise.random_numbers_host buffer.
    # This buffer has no ghost cells.
    cell_id_x = int(np.floor(drifter_pos[0]/sim.dx))
    cell_id_y = int(np.floor(drifter_pos[1]/sim.dy))
    
    # 1) Draw \tilde{\xi} \sim N(0, I)
    sim.small_scale_model_error.generateNormalDistributionCPU()
    if debug: print "noise shape: ", sim.small_scale_model_error.random_numbers_host.shape    
    
    # 1.5) Find gamma, which is needed by step 3
    gamma = np.sum(sim.small_scale_model_error.random_numbers_host **2)
    if debug: print "Gamma obtained from standard gaussian: ", gamma
    
    # 2) Apply the local sqrt(SVD)-term
    _apply_local_SVD_to_global_xi(local_sqrt_term, sim.small_scale_model_error.random_numbers_host, nx, ny, cell_id_x, cell_id_y)
    
    # 3 and 4) Apply SOAR and geostrophic balance
    H_mid = sim.downloadBathymetry()[0]
    p_eta, p_hu, p_hv = sim.small_scale_model_error._obtainOceanPerturbations_CPU(H_mid, sim.f, sim.coriolis_beta, sim.g)
    
    #gamma_from_p = np.sum(p_eta[1:-1, 1:-1]**2) + np.sum(p_hu**2) + np.sum(p_hv**2)
    #if debug: print "Gamma obtained from P^1/2 xi: ", gamma_from_p
    
    return p_eta[1:-1, 1:-1], p_hu, p_hv, gamma
    
    
def drawFromQ(S, drifter_pos, sim, Hi, debug=False):
    nx, ny = sim.nx, sim.ny
    
    # 1) Allocate memory and prepare stuff
        
    # Allocate data
    q_eta = np.zeros((ny, nx))
    q_hu  = np.zeros((ny, nx))
    q_hv  = np.zeros((ny, nx))
    
    sim.small_scale_model_error.perturbOceanStateCPU(q_eta, q_hu, q_hv, Hi, sim.f)
    
    return q_eta, q_hu, q_hv
    
    
observed_drifter_position = ensemble.observeTrueDrifters()
print observed_drifter_position

xi = [None]*ensemble.getNumParticles()


useQinsteadofP = False

if useQinsteadofP:
    for p in range(ensemble.getNumParticles()):
        q_eta, q_hu, q_hv = drawFromQ(S,  observed_drifter_position, \
                                      ensemble.particles[p], Hi, debug=False)
        xi[p] = [q_eta, q_hu, q_hv, q_x, q_y]
        print "Done with " + str(p)
        if p == 0:
            showMatrices(q_eta, q_hu, "Drawing from Q", q_hv)
else:
    for p in range(ensemble.getNumParticles()):
        #for p in range(1):
        p_eta, p_hu, p_hv, gamma = drawFromP(S, local_sqrt_term, ensemble.particles[p], 
                                             observed_drifter_position, waterDepth, debug=True)
        xi[p] = [p_eta, p_hu, p_hv, gamma]
        if p == 0:
            showMatrices(p_eta, p_hu, "Drawing from P", p_hv)
print "Done"

In [ ]:
print sim.small_scale_model_error.soar_q0

In [ ]:
for p in range(ensemble.getNumParticles()):
    showMatrices(xi[p][0], xi[p][1], "particle " + str(p), xi[p][2])

## Step 3

We now use the samples of $\xi$ to push each particle towards the observation.

In [ ]:
def implicitEquation(alpha, gamma, Nx, a):
    return (alpha-1.0)*gamma - Nx*np.log(alpha) + a

def pushParticleTowardsObservation(sim, S, \
                                   observed_drifter_position, innovation, xi, gamma, 
                                   const_H, 
                                   target_weight, w_rest, debug=True, returnKalmanGainTerm=False):
    # Following the 3rd step of the IEWPF algorithm
    
    if debug: print "(nx, ny, dx, dy): ", (sim.nx, sim.ny, sim.dx, sim.dy)
    if debug: print "observed_drifter_position: ", observed_drifter_position
    if debug: print "innovation:  ", innovation
    if debug: print "target_weight: ", target_weight

        
    # 0) Define constants/buffers etc
    geo_balance_const = sim.g*const_H/sim.f
    Nx = nx*ny*3.0 # State dimension
    
    # 0.1) Find the cell index assuming no ghost cells
    cell_id_x = int(np.floor(observed_drifter_position[0]/sim.dx))
    cell_id_y = int(np.floor(observed_drifter_position[1]/sim.dy))
    if debug: print "(cell_id_x, cell_id_y): ", (cell_id_x, cell_id_y)
    
    # 1) Solve linear problem
    e = np.dot(S, innovation)
    if debug: print "e: ", e
    
    # 2) K = QH^T e = U_GB Q^{1/2} Q^{1/2} U_GB^T  H^T e
    #    Obtain the Kalman gain
    # 2.1) U_GB^T H^T e
    # 2.1.1) H^T: The transpose of the observation operator now maps the velocity at the 
    #        drifter position to the complete state vector:
    #        H^T [hu(posx, posy), hv(posx, posy)] = [zeros_eta, 0 0 hu(posx, posy) 0 0, 0 0 hv(posx, posy) 0 0]
    
    # 2.1.2) U_GB^T: map out to laplacian stencil
    local_huhv = np.zeros(4) # representing [north, east, south, west] positions from 
    north_east_south_west_index = [[4,3,0], [3,4,1], [2,3,2], [3,2,3]] #[[y-index-eta, x-index-eta, index-local_eta_soar]]
    # the x-component of the innovation spreads to north and south
    local_huhv[0] = -e[0,0]*geo_balance_const/(2*sim.dy) # north 
    local_huhv[2] =  e[0,0]*geo_balance_const/(2*sim.dy) # south
    # the y-component of the innovation spreads to east and west
    local_huhv[1] =  e[0,1]*geo_balance_const/(2*sim.dx) # east
    local_huhv[3] = -e[0,1]*geo_balance_const/(2*sim.dx) # west
    
    # 2.1.3) Q^{1/2}:
    local_eta = np.zeros((7,7))
    for j,i,soar_res_index in north_east_south_west_index:
        if debug: print (j,i), soar_res_index
        for b in range(j-2, j+3):
            for a in range(i-2, i+3):
                local_eta[b,a] += local_huhv[soar_res_index]*SOAR_Q(a, b, i, j, sim.dx, sim.dy, 
                                                                    sim.small_scale_model_error.soar_q0, 
                                                                    sim.small_scale_model_error.soar_L)
    if debug: showMatrices(local_eta, local_eta, "local $\eta$ from global $\eta$")
           
    # 2.2) Apply U_GB Q^{1/2} to the result
    # 2.2.1)  Easiest way: map local_eta to a global K_eta_tmp buffer
    K_eta_tmp = np.zeros((sim.ny, sim.nx))
    if debug: print "K_eta_tmp.shape",  K_eta_tmp.shape
    for j in range(7):
        j_global = (cell_id_y-3+j+sim.ny)%sim.ny 
        for i in range(7):
            i_global = (cell_id_x-3+i+sim.nx)%sim.nx 
            K_eta_tmp[j_global, i_global] += local_eta[j,i]
            #K_eta_tmp[j_global, i_global] += 10000000*local_eta[j,i]
    if debug: showMatrices(K_eta_tmp, local_eta, "global K_eta from local K_eta, halfway in the calc.")
               
    # 2.2.2) Use K_eta_tmp as the noise.random_numbers_host
    sim.small_scale_model_error.random_numbers_host = K_eta_tmp
    
    # 2.2.3) Apply soar + geo-balance
    K_eta , K_hu, K_hv = sim.small_scale_model_error._obtainOceanPerturbations_CPU(ensemble.base_H, sim.f, sim.coriolis_beta, sim.g)
    K_eta = K_eta[1:-1, 1:-1]
    if debug: showMatrices(K_eta, K_hu, "Kalman gain", K_hv)
    
    if returnKalmanGainTerm:
        return K_eta, K_hu, K_hv
    
    # 3) Obtain phi = d^T * e
    phi = innovation[0]*e[0,0] + innovation[1]*e[0,1]
    if debug: print "phi: ", phi
        
    # 4) Obtain x_a
    nudged_eta, nudged_hu, nudged_hv = sim.download(interior_domain_only=True)
    original_hu = nudged_hu[cell_id_y, cell_id_x]
    original_hv = nudged_hv[cell_id_y, cell_id_x]
    if debug: showMatrices(nudged_eta, nudged_hu, "$M(x)$", nudged_hv)
    nudged_eta += K_eta
    nudged_hu += K_hu
    nudged_hv += K_hv
    if debug: print "Shapes of nudged vars: ", nudged_eta.shape, nudged_hu.shape, nudged_hv.shape
    if debug: showMatrices(nudged_eta, nudged_hu, "$x_a = M(x) + K$", nudged_hv)
        
    # 5) obtain gamma
    if debug: print "Shapes of xi: ", xi[0].shape, xi[1].shape, xi[2].shape
    #gamma = 0.0
    #for field in range(3):
    #    for j in range(ny):
    #        for i in range(nx):
    #            gamma += xi[field][j,i]*xi[field][j,i]
    if debug: print "gamma: ", gamma
    if debug: print "Nx: ", Nx
    if debug: print "w_rest: ", w_rest
    if debug: print "target_weight: ", target_weight
        
    # 6) Find a
    a = phi - w_rest + target_weight
    if debug: print "a = phi - w_rest + target_weight: ", a
            
    # 7) Solving the Lambert W function
    #alpha = 10000
    lambert_W_arg = -(gamma/Nx)*np.exp(a/Nx)*np.exp(-gamma/Nx)
    alpha_min1 = -(Nx/gamma)*np.real(lambertw(lambert_W_arg, k=-1))
    alpha_zero = -(Nx/gamma)*np.real(lambertw(lambert_W_arg))
    if debug: print "Check a against the Lambert W requirement: ", a, " < ", - Nx + gamma - Nx*np.log(gamma/Nx), " = ", a <  - Nx + gamma - Nx*np.log(gamma/Nx)
    if debug: print "-e^-1 < z < 0 : ", -1.0/np.exp(1), " < ", lambert_W_arg, " < ", 0, " = ", \
        (-1.0/np.exp(1) < lambert_W_arg, lambert_W_arg < 0)
    if debug: print "Obtained (alpha k=-1, alpha k=0): ", (alpha_min1, alpha_zero)
    if debug: print "The two branches from Lambert W: ", (lambertw(lambert_W_arg), lambertw(lambert_W_arg, k=-1))
    if debug: print "The two branches from Lambert W: ", (np.real(lambertw(lambert_W_arg)), np.real(lambertw(lambert_W_arg, k=-1)))
    
    alpha = alpha_zero
    if lambert_W_arg > (-1.0/np.exp(1)) :
        alpha_u = np.random.rand()
        if alpha_u < 0.5:
            alpha = alpha_min1
            if debug: print "Drew alpha from -1-branch"
    else:
        print "!!!!!!!!!!!!"
        print "BAD BAD ARGUMENT TO LAMBERT W"
        print "Obtained (alpha k=0, alpha k=-1): ", (alpha_zero, alpha_min1)
        print "!!!!!!!!!!!!"
    #if debug: print "Drawing random number alpha_u: ", alpha_u
    #oldDebug = debug
    #debug = True
    if debug: print "--------------------------------------"
    if debug: print "Obtained (lambert_ans k=0, lambert_ans k=-1): ", (lambertw(lambert_W_arg), lambertw(lambert_W_arg, k=-1))
    if debug: print "Obtained (alpha k=0, alpha k=-1): ", (alpha_zero, alpha_min1)
    if debug: print "Checking implicit equation with alpha (k=0, k=-1): ", \
        (implicitEquation(alpha_zero, gamma, Nx, a), implicitEquation(alpha_min1, gamma, Nx, a))
    #if debug: print "Chose alpha = ", alpha
    #if debug: print "The implicit equation looked like: \n\t" + \
    #    "(alpha - 1)"+str(gamma)+" - " + str(Nx) + "log(alpha) + " + str(a) + " = 0"
    #if debug: print "Parameters: (gamma, Nx, aj)", (gamma, Nx, a)
    
    alpha = np.sqrt(alpha)
    if debug: print "alpha = np.sqrt(alpha) ->  ", alpha
    if debug: print "--------------------------------------"
    #debug = oldDebug
    
    # 7.2) alpha*xi
    if debug: showMatrices(alpha*xi[0], alpha*xi[1], "alpha * xi", alpha*xi[2])
    
    # 8) Final nudge!
    nudged_eta += alpha*xi[0]
    nudged_hu += alpha*xi[1]
    nudged_hv += alpha*xi[2]
    if debug: showMatrices(nudged_eta, nudged_hu, "Nudged field", nudged_hv)
    if debug: print "Original velocity under drifter (hu, hv): ", (original_hu, original_hu)
    final_hu = nudged_hu[cell_id_y, cell_id_x]
    final_hv = nudged_hv[cell_id_y, cell_id_x]
    if debug: print "Final nudged velocity under drifter (hu, hv): ", (final_hu, final_hv)
    if debug: print "Diff original - nudged: ", (original_hu-final_hu, original_hu-final_hv)
                
    return nudged_eta, nudged_hu, nudged_hv

print "(ensemble.ny, ensemble.nx)", (ensemble.ny, ensemble.nx)
print "target_weight: ", target_weight
print "observation of true state: ", ensemble.observeTrueState()
innovations = ensemble.getInnovations()
w_rest = -np.log(ensemble.getGaussianWeight())
print w_rest
for p in range(ensemble.getNumParticles()):
    print " "
    n_eta, n_hu, n_hv = pushParticleTowardsObservation(ensemble.particles[p], S, \
                                                       observed_drifter_position, innovations[p], xi[p], xi[p][3], 
                                                       waterDepth, 
                                                       target_weight, w_rest[p], debug=(p==-1))

In [ ]:
def expand_to_periodic_boundaries(interior, ghostcells):
    if ghostcells == 0:
        return interior
    (ny, nx) = interior.shape
    
    nx_halo = nx + 2*ghostcells
    ny_halo = ny + 2*ghostcells
    newBuf = np.zeros((ny_halo, nx_halo))
    newBuf[ghostcells:-ghostcells, ghostcells:-ghostcells] = interior 
    for g in range(ghostcells):
        newBuf[g, :] = newBuf[ny_halo - 2*ghostcells + g, :]
        #newBuf[ny_halo - 2*ghostcells + g, :] *=0
        newBuf[ny_halo - 1 - g, :] = newBuf[2*ghostcells - 1 - g, :]
        #newBuf[2*ghostcells - 1 - g, :] *=0
    for g in range(ghostcells):
        newBuf[:, g] = newBuf[:, nx_halo - 2*ghostcells + g]
        newBuf[:, nx_halo - 1 - g] = newBuf[:, 2*ghostcells - 1 - g]
    return newBuf
    
test = np.zeros((30, 20))
for y in range(30):
    for x in range(20):
        test[y, x] = np.sqrt((x - 9.5)**2 + (y - 14.5)**2)
fig = plt.figure(figsize=(4,4))
plt.imshow(test, interpolation="None")
test = expand_to_periodic_boundaries(test, 2)
fig = plt.figure(figsize=(4,4))
plt.imshow(test, interpolation="None")


# Testing IEWPF

Here, we make a test of the entire IEWPF algorithm applied to a suitable test case.

In [ ]:
# DEFINE PARAMETERS

#Coriolis well balanced reconstruction scheme
nx = 40
ny = 40

dx = 4.0
dy = 4.0

dt = 0.05*3
g = 9.81
r = 0.0

f = 0.05
beta = 0.0

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

# Define which cell index which has lower left corner as position (0,0)
x_zero_ref = 2
y_zero_ref = 2

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
dataShapeHi = (ny + ghosts[0]+ghosts[2]+1, 
             nx + ghosts[1]+ghosts[3]+1)

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
eta0_extra = np.zeros(dataShape, dtype=np.float32, order='C')
hv0 = np.zeros(dataShape, dtype=np.float32, order='C');
hu0 = np.zeros(dataShape, dtype=np.float32, order='C');
waterDepth = 1.0
Hi = np.ones(dataShapeHi, dtype=np.float32, order='C')*waterDepth

# Add disturbance:
initOption = 3
if initOption == 1:
    # Original initial conditions
    rel_grid_size = nx*1.0/dx
    BC.addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05*rel_grid_size, validDomain)
    eta0 = eta0*0.3
    BC.addBump(eta0, nx, ny, dx, dy, 0.7, 0.3, 0.10*rel_grid_size, validDomain)
    eta0 = eta0*(-1.3)
    BC.addBump(eta0, nx, ny, dx, dy, 0.15, 0.8, 0.03*rel_grid_size, validDomain)
    eta0 = eta0*1.0
    BC.addBump(eta0, nx, ny, dx, dy, 0.6, 0.75, 0.06*rel_grid_size, validDomain)
    BC.addBump(eta0, nx, ny, dx, dy, 0.2, 0.2, 0.01*rel_grid_size, validDomain)
    eta0 = eta0*(-0.03)
    BC.addBump(eta0_extra, nx, ny, dx, dy, 0.5, 0.5, 0.4*rel_grid_size, validDomain)
    eta0 = eta0 + 0.02*eta0_extra
    BC.initializeBalancedVelocityField(eta0, Hi, hu0, hv0, f, beta, g, nx, ny, dx ,dy, ghosts)
    eta0 = eta0*0.5
elif initOption == 2:
    # Initial conditions used for the SIR filter
    rel_grid_size = nx*1.0/dx
    BC.addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05*rel_grid_size, validDomain)
    eta0 = eta0*0.3
    BC.addBump(eta0, nx, ny, dx, dy, 0.7, 0.3, 0.10*rel_grid_size, validDomain)
    eta0 = eta0*(-1.3)
    BC.addBump(eta0, nx, ny, dx, dy, 0.15, 0.8, 0.03*rel_grid_size, validDomain)
    eta0 = eta0*1.0
    BC.addBump(eta0, nx, ny, dx, dy, 0.6, 0.75, 0.06*rel_grid_size, validDomain)
    BC.addBump(eta0, nx, ny, dx, dy, 0.2, 0.2, 0.01*rel_grid_size, validDomain)
    eta0 = eta0*(-0.03)
    BC.addBump(eta0_extra, nx, ny, dx, dy, 0.5, 0.5, 0.4*rel_grid_size, validDomain)
    eta0 = eta0 + 0.02*eta0_extra
    BC.initializeBalancedVelocityField(eta0, Hi, hu0, hv0, f, beta, g, nx, ny, dx ,dy, ghosts)
    eta0 = eta0*0.5
elif initOption == 3:
    # Initial conditions random - see further down!
    pass
    

if 'sim' in globals():
    sim.cleanUp()
if 'ensemble' in globals():
    ensemble.cleanUp()
    
q0 = 0.5*dt*f/(g*waterDepth)
print "q0: ", q0
print "[f, g, H, dt]", [f, g, waterDepth, dt]
print "f/gH: ", f/(g*waterDepth)
print "gH/f: ", g*waterDepth/f

reload(CDKLM16)
reload(BaseOceanStateEnsemble)
reload(OceanNoiseEnsemble)
reload(PlotHelper)
reload(dautils)
sim = CDKLM16.CDKLM16(cl_ctx, eta0, hu0, hv0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions, \
                      write_netcdf=False, \
                      small_scale_perturbation=True, \
                      small_scale_perturbation_amplitude=q0)
if initOption == 3:
    sim.perturbState(q0_scale=50)
    
ensemble_size = 30

ensemble = OceanNoiseEnsemble.OceanNoiseEnsemble(ensemble_size, cl_ctx,  
                                                 observation_type=dautils.ObservationType.DirectUnderlyingFlow)
ensemble.setGridInfoFromSim(sim)
ensemble.setStochasticVariables(#observation_variance_factor=2.0,
                                observation_variance = 0.01**2,
                                small_scale_perturbation_amplitude=q0)
                                #initialization_variance_factor_ocean_field=50)
ensemble.init()

max_dt = ensemble.findLargestPossibleTimeStep()
print "max_dt: ", max_dt

S = createS(ensemble, waterDepth)
local_SVD_block = generateLocaleSVDforP(ensemble, S)
infoPlots = []
debug=False

fig = plt.figure()
plotter = PlotHelper.EnsembleAnimator(fig, ensemble, trueStateOnly=False)

def keepPlot(ensemble, infoPlots, it, stage):
    title = "it=" + str(it) + " before IEWPF"
    if stage == 2:
        title = "it=" + str(it) + " during IEWPF (with deterministic step)"
    elif stage == 3:
        title = "it=" + str(it) + " after IEWPF"
    infoFig = ensemble.plotDistanceInfo(title=title, printInfo=False)
    plt.close(infoFig)
    infoPlots.append(infoFig)
    
def iewpf(ensemble, infoPlots, it):
    
    # Step -1: Deterministic step
    t = ensemble.step_truth(dt, stochastic=True)
    
    # Step 0: Obtain distances
    observed_drifter_position = ensemble.observeTrueDrifters()
    innovations = ensemble.getInnovations()
    #w_rest = -np.log(ensemble.getGaussianWeight())
    w_rest = -np.log(1.0/ensemble.getNumParticles())*np.ones(ensemble.getNumParticles())
    t = ensemble.step_particles(dt, stochastic=False)
    
    # save plot halfway
    keepPlot(ensemble, infoPlots, it, 1)
    
    # Step 1: Find maximum weight
    target_weight = obtainTargetWeight(ensemble, S, w_rest)
    #print "WWWWWWWWWWWWWWW"
    #print "Target weight: ", target_weight
    #print "-log(target_weight): ", -np.log(target_weight)
    #print "exp(-target_weight): ", np.exp(-target_weight)
    #print "1/Ne: ", 1.0/ensemble.getNumParticles()
    #print "WWWWWWWWWWWWWWW"
    
    
    for p in range(ensemble.getNumParticles()):

        # Step 2: Sample xi Ìƒ N(0, P)
        p_eta, p_hu, p_hv, gamma = drawFromP(S, local_SVD_block, ensemble.particles[p], 
                                             observed_drifter_position, waterDepth, debug=False)
        xi = [p_eta, p_hu, p_hv] 

        # Step 3: Pull particles towards observation
        #w_rest[p] = 0.0
        new_eta, new_hu, new_hv = pushParticleTowardsObservation(ensemble.particles[p], S, \
                                                                 observed_drifter_position, 
                                                                 innovations[p], xi, 
                                                                 gamma,
                                                                 waterDepth, target_weight, 
                                                                 w_rest[p], False)
        new_eta = expand_to_periodic_boundaries(new_eta, 2)
        new_hu  = expand_to_periodic_boundaries(new_hu,  2)
        new_hv  = expand_to_periodic_boundaries(new_hv,  2)
        ensemble.particles[p].upload(new_eta, new_hu, new_hv)
        #ensemble.particles[p].drifters.setDrifterPositions(newPos)
        
    # save plot after
    keepPlot(ensemble, infoPlots, it, 3)


T = 13
sub_t = 10*dt
#observation_iterations = [4, 8, 12, 16]
observation_iterations = range(1, 25, 2)
def animate(i):
    if (i>0):
        t = ensemble.step(sub_t)
    else:
        t = 0.0

    for oi in observation_iterations:
        if i == oi:
            print "Enter IEWPF"
            max_dt = ensemble.findLargestPossibleTimeStep()
            print "max_dt: ", max_dt
            iewpf(ensemble, infoPlots, i)
            #keepPlot(ensemble, infoPlots, i, 1)
            print "Successful IEWPF (hopefully)"
        
    plotter.plot(ensemble);

    fig.suptitle("Ensemble = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%5 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t)
        #if i != 0: 
        #    ensemble.printMaxOceanStates()

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
def show_figures(figs):
    for f in figs:
        dummy = plt.figure()
        new_manager = dummy.canvas.manager
        new_manager.canvas.figure = f
        f.set_canvas(new_manager.canvas)
        filename= f._suptitle.get_text().replace(" ", "_").replace("=_", "") + ".png"
        #plt.savefig(filename)
show_figures(infoPlots)
fig = ensemble.plotDistanceInfo(title="Final ensemble")
ensemble.plotEnsemble()


***Prototyping***

## Checking that np.reshape works as expected

Assume that it reshapes by [1,2,3,4] -> [[1,2], [3,4]]

In [ ]:
test_array = np.array(range(1,21))
print test_array

# Expect 4 rows, 5 columns
test_matrix = np.reshape(test_array, (4,5))
print test_matrix
print "test_matrix.shape", test_matrix.shape
print test_matrix[3,1], test_array[3*5+1]

## Checking  $Q^{1/2} \xi$ 

Checking that $Q^{1/2}\xi$ is the same for the operations and the matrix multiplications.

$$ Q^{1/2} \xi = U_{GB} Q_{SOAR}^{1/2}\xi$$


The difference below comes from the fact that $U_{GB}$ actually depends on $\eta$... In the stencil operation, we use that 
$$\Delta hu_{i,j} = -\frac{g (H_{i,j} + \eta_{i,j})}{f}\frac{\eta_{i, j+1} - \eta_{i, j-1}}{2 \Delta y}, $$
while the matrix is constructed based on 
$$\Delta hu_{i,j} = -\frac{g H_0}{f}\frac{\eta_{i, j+1} - \eta_{i, j-1}}{2 \Delta y}, $$
assuming that $H(x,y) = H_0 + \eta \approx H_0$. 

In [ ]:
if 'noise' in globals():
    noise.cleanUp()
reload(OceanStateNoise)

NX = 6#nx
NY = 6#ny
noise = OceanStateNoise.OceanStateNoise(sim.cl_ctx, sim.cl_queue,
                                        NX, NY, dx, dy,
                                        Common.BoundaryConditions(2,2,2,2), False,
                                        soar_q0=q0)


xi_array = np.random.normal(size=NY*NX)
#xi_array = np.ones(NY*NX)
#xi_array = np.linspace(0, 10, NY*NX)
def make_bump(xi, nx, ny):
    for j in range(ny):
        for i in range(nx):
            xi_array[j*nx + i] = np.exp(-0.3*np.sqrt((j-ny/2)**2 + (i-nx/2)**2))
#make_bump(xi_array, NX, NY)
print "xi_array.shape", xi_array.shape
xi_matrix = np.reshape(xi_array, (NY, NX))
#xi_matrix = np.reshape(xi_array, (NY, NX)).transpose()
#xi_array = np.reshape(xi_matrix, NX*NY)
print "xi_matrix.shape", xi_matrix.shape
print "Hi.shape", Hi.shape, "Hi[4,4]: ", Hi[4,4]
Hi_interior = Hi[2:-2, 2:-2]
print "Hi_interior.shape", Hi_interior.shape
print "min-max Hi_interior", (np.min(Hi_interior), np.max(Hi_interior))

print "------------- Starting CPU operations --------"
print "noise.random_numbers_host.shape", noise.random_numbers_host.shape
noise.random_numbers_host = xi_matrix
res_eta, res_hu, res_hv = noise._obtainOceanPerturbations_CPU(Hi_interior, sim.f,
                                                              sim.coriolis_beta, sim.g)
res_eta = res_eta[1:-1, 1:-1]
showMatrices(res_eta, res_hu, "Operations from CPU", res_hv)

print "-------- Starting matrices -----------"
Q_soar = _createCutoffSOARMatrixQ(ensemble, nx=NX, ny=NY, cutoff=2)
U_GB = _createUGBmatrix(ensemble, nx=NX, ny=NY)
print "Q_soar.shape", Q_soar.shape
print "U_GB.shape", U_GB.shape
#fig = plt.figure(figsize=(6,6))
#plt.imshow(_createUGBmatrix(ensemble, nx=4, ny=5), interpolation="None")

print "xi_array.shape", xi_array.shape
#Qxi = xi_array #
Qxi = np.dot(Q_soar, xi_array)
print "Qxi.shape", Qxi.shape
UQxi = np.dot(U_GB, Qxi)
print "UQxi.shape", UQxi.shape

res_eta_matrix = np.reshape(UQxi[0,       :  NX*NY], (NY, NX))
res_hu_matrix  = np.reshape(UQxi[0,  NX*NY:2*NX*NY], (NY, NX))
res_hv_matrix  = np.reshape(UQxi[0,2*NX*NY:       ], (NY, NX))
showMatrices(res_eta_matrix, res_hu_matrix, "Matrix operations", res_hv_matrix)

abs_diff_eta = np.abs(res_eta_matrix - res_eta)
abs_diff_hu  = np.abs(res_hu_matrix  - res_hu )
abs_diff_hv  = np.abs(res_hv_matrix  - res_hv )
showMatrices(abs_diff_eta, abs_diff_hu, "Absolute differences", abs_diff_hv)



## Checking  $P^{1/2} \xi$ 

Checking that $P^{1/2}\xi$ is the same for the operations and the matrix multiplications.

$$ Q^{1/2} \xi = U_{GB} \tilde{Q}^{1/2} U \Sigma^{1/2} \tilde{\xi}$$

## Checking the Kalman gain term
Comparing the Kalman gain term by finding it as operations, and by finding it through matrix operations.

The Kalman gain is given by
$$K = Q H^T S d,$$
which boils down to
$$ K = U_{GB} \tilde{Q}^{1/2} \tilde{Q}^{1/2} U_{GB}^T H^T S d$$

In [ ]:
innovations = ensemble.getInnovations()
print "innovations.shape", innovations.shape
observed_drifter_position = ensemble.observeTrueState()[0:2]

K_eta, K_hu, K_hv = pushParticleTowardsObservation(ensemble.particles[2], S, \
                                                   observed_drifter_position, innovations[2], \
                                                   xi[2], xi[2][3], 
                                                   waterDepth, 
                                                   target_weight, w_rest[2], debug=False, \
                                                   returnKalmanGainTerm=True)
showMatrices(K_eta, K_hu, "Kalman gains from operations", K_hv)

print "----------- Starting matrix operatioins -----------------"
cell_id_x = int(np.floor(observed_drifter_position[0]/ensemble.dx))
cell_id_y = int(np.floor(observed_drifter_position[1]/ensemble.dy))
Q_soar = _createCutoffSOARMatrixQ(ensemble)
U_GB = _createUGBmatrix(ensemble)
H = _createMatrixH(nx, ny, cell_id_x, cell_id_y)

print "Q_soar.shape", Q_soar.shape
print "U_GB.shape", U_GB.shape
print "H.shape", H.shape
print "S.shape", S.shape

print "innovations[2]", innovations[2]
print "innovations[2].shape", np.array(innovations[2]).shape
d = np.array([[innovations[2,0]], [innovations[2,1]]])
print "d", d
print "d.shape", d.shape

# K = Q H^T S d
#   = U_GB Q_soar Q_soar U_GB^T H^T S d
e = np.dot(S, d)
print "e", e
HTe = np.dot(H.transpose(), e)
print "HTe.shape", HTe.shape

UTHTe = np.dot(U_GB.transpose(), HTe)
QUTHTe = np.dot(Q_soar.transpose(), UTHTe)
halfway = np.reshape(QUTHTe, (ny, nx))
#showMatrices(halfway, halfway, "halfway matrix", halfway)
QQUTHTe = np.dot(Q_soar, QUTHTe)
K = np.dot(U_GB, QQUTHTe)

print "K.shape", K.shape
K_eta_matrix = np.reshape(K[       :nx*ny  ], (ny, nx))
K_hu_matrix  = np.reshape(K[  nx*ny:2*nx*ny], (ny, nx))
K_hv_matrix  = np.reshape(K[2*nx*ny:       ], (ny, nx))
showMatrices(K_eta_matrix, K_hu_matrix, "Kalman gains from matrices", K_hv_matrix)

K_eta_abs_diff = np.abs(K_eta_matrix - K_eta)
K_hu_abs_diff  = np.abs( K_hu_matrix - K_hu )
K_hv_abs_diff  = np.abs( K_hv_matrix - K_hv )
showMatrices(K_eta_abs_diff, K_hu_abs_diff, "K abs differences", K_hv_abs_diff)



#######################

eta, hu, hv = ensemble.particles[2].download(interior_domain_only=True)
true_eta, true_hu, true_hv = ensemble.particles[ensemble.obs_index].download(interior_domain_only=True)

print "-----------------------------------"
print "Observation:          ", ensemble.observeTrueState()
print "Observed particle pos:", observed_drifter_position
print "True velocity:        ", [true_hu[cell_id_y, cell_id_x], true_hv[cell_id_y, cell_id_x]]
print "Particle velocity:    ", [hu[cell_id_y, cell_id_x], hv[cell_id_y, cell_id_x]]
print "Ocean depth:          ", np.max(ensemble.base_H)

print "---------"
print "innovation:           ", innovations[2]
#print "y - particle velocity:", [
print "stencil [K_hu, K_hv]: ", [K_hu[cell_id_y, cell_id_x], K_hv[cell_id_y, cell_id_x]]
print "matrix  [K_hu, K_hv]: ", [K_hu_matrix[cell_id_y, cell_id_x], K_hv_matrix[cell_id_y, cell_id_x]]
print "                 H*K: ", np.dot(H, K)





# Investigating the innermost term of $P$

Step 2 of the IEWPF algorithm consists of drawing $\xi \sim N(0, P)$, meaning that we have to find $\xi = P^{1/2} \tilde{\xi}$, in which $\tilde{\xi} \sim N(0, I)$ and $P$ is given by
$$ P = Q - Q H^T (HQH^T + R)^{-1} H Q \\
	  \qquad = Q^{1/2}(I - Q^{1/2, T}H^T S H Q^{1/2})Q^{1/2,T} \\
      \qquad = Q^{1/2}(I - \tilde{P})Q^{1/2,T} $$
In order to find $P^{1/2}$, we need to express $(I - \tilde{P})^{1/2}$. This will be done through a singular value decomposition of $(I - \tilde{P})$.

By considering the local structure of $Q^{1/2}$ and the size of the size of the observation operator $H$, we see that $\tilde{P}$ will be matrix which is zero everywhere, except in a $7 \times 7$ neighbour block, where the location of the block depends on the position of the observed drifter. Since $H$ will change, $P$ is subject to change as well, but the structure of $P$ will be constant. We can therefore obtain $P^{1/2}$ based on the SVD of the non-zero block of $(I-\tilde{P})$. 


The aim of the code below is to obtain the full matrix $\tilde{P}$, and confirm that we can calculate the SVD of $(I - \tilde{P})$.
By using 
$$(I - \tilde{P}) = U \Sigma V^T,$$ 
where $\Sigma$ is a diagonal matrix, we now get 
$$P = Q^{1/2} U \Sigma^{1/2} \Sigma^{1/2} V^T Q^{1/2, T} \\
	  \quad \qquad = U_{GB} \tilde{Q}^{1/2} U \Sigma^{1/2} \Sigma^{1/2} V^T \tilde{Q}^{1/2} U_{GB}^T$$

Using the convention $P = P^{1/2} P^{1/2, T}$, we get a $\xi \sim N(0, P)$ as
$$ \xi  = U_{GB} \tilde{Q}^{1/2} U \Sigma^{1/2} \tilde{\xi}$$


In [ ]:
def periodic_SOAR_Q(a_x, a_y, b_x, b_y, dx, dy, nx, ny, q0, L):
    dist_x = min((a_x - b_x)**2, (a_x - (b_x + nx))**2, (a_x - (b_x - nx))**2)
    dist_y = min((a_y - b_y)**2, (a_y - (b_y + ny))**2, (a_y - (b_y - ny))**2)
    
    dist = np.sqrt( dx*dx*dist_x  +  dy*dy*dist_y)
    
    return q0*(1.0 + dist/L)*np.exp(-dist/L)

def createCutoffMatrixQ(nx, ny, dx=1.0, dy=1.0, q0=1.0, L=1.0, cutoff=2):
    Q = np.zeros((ny*nx, ny*nx))
    for a_y in range(ny):
        for a_x in range(nx):
            j = a_y*nx + a_x
            for b_y in range(a_y-cutoff, a_y+cutoff+1):
                if b_y < 0:    
                     b_y = b_y + ny
                if b_y > ny-1: 
                    b_y = b_y - ny
                for b_x in range(a_x-cutoff, a_x+cutoff+1):
                    if b_x < 0:
                        b_x = b_x + nx
                    if b_x > nx-1: 
                        b_x = b_x - nx
                    i = b_y*nx + b_x
                    Q[j, i] = periodic_SOAR_Q(a_x, a_y, b_x, b_y, dx, dy, nx, ny, q0, L)
    return Q

def createFullMatrixQ(nx, ny, dx=1, dy=1, q0=1, L=1):
    Q = np.zeros((ny*nx, ny*nx))
    for a_y in range(ny):
        for a_x in range(nx):
            j = a_y*nx + a_x
            for b_y in range(ny):
                for b_x in range(nx):
                    i = b_y*nx + b_x
                    Q[j, i] = periodic_SOAR_Q(a_x, a_y, b_x, b_y, dx, dy, nx, ny, q0, L)
    return Q

nx = 10
ny = 10
dx = ensemble.dx
dy = ensemble.dy
L = 0.75*dx
g = ensemble.g
q0 = ensemble.small_scale_perturbation_amplitude
Q = createCutoffMatrixQ(nx, ny, dx, dy, q0=q0, L=L)
fig = plt.figure(figsize=(4,4))
plt.imshow(Q, interpolation="None")
plt.title("SOAR Q (cutoff)")
plt.colorbar()
fig = plt.figure(figsize=(4,4))
plt.imshow(Q == 0.0, interpolation="None")

#fullQ = createFullMatrixQ(nx, ny, L=L, q0=q0)
#fig = plt.figure(figsize=(4,4))
#plt.imshow(fullQ, interpolation="None")
#plt.title("SOAR Q (full)")
#plt.colorbar()
#fig = plt.figure(figsize=(4,4))
#plt.imshow(Q - fullQ, interpolation="None")
#plt.colorbar()
#plt.title("SOAR Q (cutoff - diff)")



def createUGBmatrix(nx, ny, g=9.81, H=10, f=30, dx=1.0, dy=1.0):
    I = np.eye(nx*ny)
    A_hu = np.zeros((ny*nx, ny*nx))
    A_hv = np.zeros((ny*nx, ny*nx))
    for a_y in range(ny):
        for a_x in range(nx):
            i = a_y*nx + a_x
            
            # geo balance for hu:
            j = (a_y+1)*nx + a_x
            if a_y == ny-1:
                j = 0*nx + a_x
            A_hu[j,i] = 1.0
            j = (a_y-1)*nx + a_x
            if a_y == 0:
                j = (ny-1)*nx + a_x
            A_hu[j,i] = -1.0
            
            # geo balance for hv:
            j = a_y*nx + a_x + 1
            if a_x == nx-1:
                j = a_y*nx + 0
            A_hv[j,i] = 1.0
            
            j = a_y*nx + a_x - 1
            if a_x == 0:
                j = a_y*nx + nx - 1
            A_hv[j,i] = -1.0
            
    A_hu *= -g*H/(f*2*dx)
    A_hv *=  g*H/(f*2*dy)
            
    return np.bmat([[I], [A_hu], [A_hv]])


    
U = createUGBmatrix(nx, ny, g=g, H=waterDepth, f=ensemble.f, dx=dx, dy=dy)
fig = plt.figure(figsize=(4,8))
plt.imshow(U, interpolation="None")
plt.title("U")
plt.colorbar()


S = np.matrix([[2.0, 0.0], [0.0, 2.0]])
fig = plt.figure(figsize=(2,2))
plt.imshow(S, interpolation="None")
plt.title("S")
plt.colorbar()




In [ ]:

pos_x, pos_y = 5, 5

def createMatrixH(nx, ny, pos_x, pos_y):
    H = np.zeros((2, 3*nx*ny))
    index = pos_y*nx + pos_x
    H[0, 1*nx*ny + index] = 1
    H[1, 2*nx*ny + index] = 1
    return H

H = createMatrixH(nx, ny, pos_x, pos_y)
print "H.shape", H.shape
fig = plt.figure(figsize=(20,2))
plt.imshow(H, interpolation="None", cmap="cool")
plt.title("H")
plt.colorbar()

Using $Q = U_{GB} \tilde{Q}^{1/2} \tilde{Q}^{1/2} U_{GB}^T$, where $\tilde{Q}^{1/2}$ is symmetric, we get 
$$\tilde{P} = \tilde{Q}^{1/2} U_{GB}^T H^T S H U_{GB} \tilde{Q}^{1/2}$$

We go through the computations ofÂ $\tilde{P}$ step by step

In [ ]:
UQ = np.matmul(U, Q)
fig = plt.figure(figsize=(4,4))
plt.imshow(UQ, interpolation="None")
plt.title("UQ")
plt.colorbar()


In [ ]:
HUQ = np.matmul(H, UQ)
fig = plt.figure(figsize=(20,2))
plt.imshow(HUQ, interpolation="None")
plt.title("HUQ")
plt.colorbar()

In [ ]:
SHUQ = np.matmul(S, HUQ)
fig = plt.figure(figsize=(20,2))
plt.imshow(SHUQ, interpolation="None")
plt.title("SHUQ")
plt.colorbar()

In [ ]:
HTSHUQ = np.matmul(H.transpose(), SHUQ)
fig = plt.figure(figsize=(4,4))
plt.imshow(HTSHUQ, interpolation="None")
plt.title("HTSHUQ")
plt.colorbar()


In [ ]:
UTHTSHUQ = np.matmul(U.transpose(), HTSHUQ)
fig = plt.figure(figsize=(4,4))
plt.imshow(UTHTSHUQ, interpolation="None")
plt.title("UTHTSHUQ")
plt.colorbar()

In [ ]:
tildeP = np.matmul(Q, UTHTSHUQ)
fig = plt.figure(figsize=(4,4))
plt.imshow(tildeP, interpolation="None")
plt.title("tilde(P) = QUTHTSHUQ")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.imshow(tildeP == 0, interpolation="None")
plt.title("tilde(P) non-zeros")
plt.colorbar()



In [ ]:
def extractNonZeroBlocks(tildeP, nx, ny, pos_x, pos_y):
    
    # The Q^{1/2} U_GB^T pattern spreads information to a 7x7 cell area, but without the corners
    # Hence, the nonzero structure of tilde{P} should be a block of 7x7-4 = 49-4 = 45 rows and cells.
    
    # Strategy: Fill 49 by 49 area
    
    tildeP_block = np.zeros((49,49))
    
    # Read the non-zero structure from tildeP to tildeP_block
    for loc_y_j in range(7):
        global_y_j = pos_y - 3 + loc_y_j
        for loc_x_j in range(7):
            global_x_j = pos_x - 3 + loc_x_j
            
            global_j = global_y_j*nx + global_x_j
            local_j = loc_y_j*7 + loc_x_j
            
            for loc_y_i in range(7):
                global_y_i = pos_y - 3 + loc_y_i
                for loc_x_i in range(7):
                    global_x_i = pos_x - 3 + loc_x_i
                    
                    global_i = global_y_i*nx + global_x_i
                    local_i = loc_y_i*7 + loc_x_i
                    
                    tildeP_block[local_j, local_i] = tildeP[global_j, global_i] 
    
    # Delete the rows and arrays that should not take part of the block
    #local_indices_to_delete = [0, 6, 49-7, 49-1]
    #tildeP_block = np.delete(tildeP_block, local_indices_to_delete, 0)
    #tildeP_block = np.delete(tildeP_block, local_indices_to_delete, 1)
    
    validate = False
    if validate:
        nz_y, nz_x = np.nonzero(tildeP)
        print len(nz_y), nz_y
        print len(nz_x), nz_x
        print 7*7*7*7 


        unique_nz_y = np.unique(nz_y)
        unique_nz_x = np.unique(nz_x)

        print len(unique_nz_y), unique_nz_y
        print len(unique_nz_x), unique_nz_x
        validation = tildeP[unique_nz_y, :]
        validation = validation[:, unique_nz_x]
        validation_nz_y, validation_nz_x = np.nonzero(validation)
        print "Num nonzeros validation:  ", len(validation_nz_y), len(validation_nz_x)

        fig = plt.figure(figsize=(4,4))
        plt.imshow(validation, interpolation="None")
        plt.title("validation for tilde(P) non-zeros block")
        plt.colorbar()

        fig = plt.figure(figsize=(4,4))
        plt.imshow(tildeP_block, interpolation="None")
        plt.title("tildeP_block")
        plt.colorbar()

        fig = plt.figure(figsize=(4,4))
        plt.imshow(tildeP_block - validation, interpolation="None")
        plt.title("tildeP_block - validation")
        plt.colorbar()
    
    return tildeP_block
    
    
tildeP_block = extractNonZeroBlocks(tildeP, nx, ny, pos_x, pos_y )
fig = plt.figure(figsize=(4,4))
plt.imshow(tildeP_block, interpolation="None")
plt.title("tilde(P) non-zeros block")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.imshow(tildeP_block == 0, interpolation="None")
plt.title("tilde(P) non-zeros block nonzeros")
plt.colorbar()

print "tildeP_block.shape", tildeP_block.shape

# Finding the Singular-Value Decomposition

Need the SVD so that 
$$ (I - \tilde{P}) = U \Sigma V^T = U \Sigma^{1/2} \Sigma^{1/2} V^T $$

In [ ]:
svd_block_input = np.eye(49) - tildeP_block
u, s, vh = np.linalg.svd(svd_block_input, full_matrices=True)

print "u: ", u.shape
print "s: ", s.shape
print "vh:", vh.shape

fig = plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.imshow(u, interpolation="None")
plt.title("u")
plt.colorbar()
plt.subplot(132)
plt.imshow(np.diag(s), interpolation="None")
plt.title("s")
plt.colorbar()
plt.subplot(133)
plt.imshow(vh, interpolation="None")
plt.title("vh")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.imshow(u - vh.transpose(), interpolation="None")
plt.title("u - vh")
plt.colorbar()

svd_block_output = np.matmul(u, np.matmul(np.diag(s), vh))
fig = plt.figure(figsize=(4,4))
plt.imshow(svd_block_output, interpolation="None")
plt.title("svd_block_output")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.imshow(svd_block_input - svd_block_output, interpolation="None")
plt.title("svd_block_input - svd_block_output")
plt.colorbar()


fig = plt.figure(figsize=(4,4))
plt.imshow(np.abs(svd_block_output) < 0.00000001, interpolation="None")
plt.title("svd_block_output non-zeros")
plt.colorbar()
fig = plt.figure(figsize=(4,4))
plt.imshow(np.abs(svd_block_input) < 0.00000001, interpolation="None")
plt.title("svd_block_input non-zeros")
plt.colorbar()


print s

In [ ]:
sqrt_term = np.matmul(u, np.diag(np.sqrt(s)))

fig = plt.figure(figsize=(4,4))
plt.imshow(sqrt_term, interpolation="None")
plt.title("sqrt_term")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.imshow(np.dot(sqrt_term, sqrt_term.transpose()) - np.eye(49), interpolation="None")
plt.title("sqrt_term * sqrt_term^T")
plt.colorbar()

onesies = np.ones(49)
onesies = np.dot(sqrt_term, onesies)
onesies = onesies.reshape((7,7))
fig = plt.figure(figsize=(4,4))
plt.imshow(onesies, interpolation="None")
plt.title("sqrt_term * onesies")

print "sqrt_term.shape", sqrt_term.shape

filename = "svd_tests/local_sqrt_term_nx_" + str(nx) + "_ny_" + str(ny) + "_posx_" + str(pos_x) + "_posy_" + str(pos_y) +".npy"
print filename
np.save(filename, sqrt_term)

## Mapping the SVD-block back to the global domain

In [ ]:
def extendSVDBlock(svd_block, nx, ny, pos_x, pos_y):
    
    # The Q^{1/2} U_GB^T pattern spreads information to a 7x7 cell area, but without the corners
    # Hence, the nonzero structure of tilde{P} should be a block of 7x7-4 = 49-4 = 45 rows and cells.
    
    # Strategy: Fill 49 by 49 area
    
    global_svd = np.eye(nx*ny)
    
    # Read the non-zero structure from tildeP to tildeP_block
    for loc_y_j in range(7):
        global_y_j = pos_y - 3 + loc_y_j
        for loc_x_j in range(7):
            global_x_j = pos_x - 3 + loc_x_j
            
            global_j = global_y_j*nx + global_x_j
            local_j = loc_y_j*7 + loc_x_j
            
            for loc_y_i in range(7):
                global_y_i = pos_y - 3 + loc_y_i
                for loc_x_i in range(7):
                    global_x_i = pos_x - 3 + loc_x_i
                    
                    global_i = global_y_i*nx + global_x_i
                    local_i = loc_y_i*7 + loc_x_i
                    
                    global_svd[global_j, global_i] = svd_block[local_j, local_i]
    return global_svd            
        
global_svd = extendSVDBlock(svd_block_output, nx, ny, pos_x, pos_y)
fig = plt.figure(figsize=(4,4))
plt.imshow(global_svd, interpolation="None")
plt.title("global_svd ")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.imshow(np.abs(global_svd) < 0.00000001, interpolation="None")
plt.title("global_svd non-zeros")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.imshow(np.abs(svd_block_output) < 0.00000001, interpolation="None")
plt.title("svd_block_output non-zeros")
plt.colorbar()

eyeMinusTildeP = np.eye(nx*ny) - tildeP
fig = plt.figure(figsize=(4,4))
plt.imshow(global_svd - eyeMinusTildeP, interpolation="None")
plt.title("global_svd - eyeMinusTildeP")
plt.colorbar()

global_sqrt_term = extendSVDBlock(sqrt_term, nx, ny, pos_x, pos_y)
fig = plt.figure(figsize=(4,4))
plt.imshow(global_sqrt_term, interpolation="None")
plt.title("global_sqrt_term ")
plt.colorbar()

# Apply local and global $U \Sigma^{1/2}$ to vector
And see that the results are the same.

In [ ]:
glob_v = np.random.rand(nx*ny)

def global_to_local_vec(glob_vec, nx, ny, pos_x, pos_y):
    
    # Strategy: go from vec nx*ny to vec 49
    
    loc_vec = np.zeros(49)
    
    # Read the non-zero structure from tildeP to tildeP_block
    for loc_y_j in range(7):
        global_y_j = pos_y - 3 + loc_y_j
        for loc_x_j in range(7):
            global_x_j = pos_x - 3 + loc_x_j
            
            global_j = global_y_j*nx + global_x_j
            local_j = loc_y_j*7 + loc_x_j
            
            loc_vec[local_j] = glob_vec[global_j]
    return loc_vec

def write_local_to_global_vec(glob_vec, loc_vec, nx, ny, pos_x, pos_y):
    
    # Write the elements in loc_vec to the appropriate locations in glob_vec
        
    # Read the non-zero structure from tildeP to tildeP_block
    for loc_y_j in range(7):
        global_y_j = pos_y - 3 + loc_y_j
        for loc_x_j in range(7):
            global_x_j = pos_x - 3 + loc_x_j
            
            global_j = global_y_j*nx + global_x_j
            local_j = loc_y_j*7 + loc_x_j
            
            glob_vec[global_j] = loc_vec[local_j]
    
# This is the function that needs to be implemented on GPU
def apply_local_SVD_to_global_eta(local_sqrt_term, global_eta, nx, ny, pos_x, pos_y):
    """
    Despite the bad name, this is a good function!
    
    It takes as input:
     - local sqrt(SVD) as U*sqrt(Sigma) in a (49, 49) buffer 
     - the global xi stored in a (ny, nx) buffer
     
    It returns the product of the first and second argument, as U*sqrt(Sigma)*xi, in a (ny, nx) buffer 
    """
    
    
    # Copy the result (representing the multiplication with I)
    res_global_eta = global_eta.copy()
    
    # Read the non-zero structure from tildeP to tildeP_block
    for loc_y_j in range(7):
        global_y_j = pos_y - 3 + loc_y_j
        for loc_x_j in range(7):
            global_x_j = pos_x - 3 + loc_x_j
            
            global_j = global_y_j*nx + global_x_j
            local_j = loc_y_j*7 + loc_x_j
            
            #loc_vec[local_j] = glob_vec[global_j]
            
            xi_j = 0.0
            for loc_y_i in range(7):
                global_y_i = pos_y - 3 + loc_y_i
                for loc_x_i in range(7):
                    global_x_i = pos_x - 3 + loc_x_i
                    
                    global_i = global_y_i*nx + global_x_i
                    local_i = loc_y_i*7 + loc_x_i
                    
                    xi_j += local_sqrt_term[local_j, local_i]*global_eta[global_y_i, global_x_i]
            
            res_global_eta[global_y_j, global_x_j] = xi_j
            
    return res_global_eta
            
# Check that the above functions works:
loc_v = global_to_local_vec(glob_v, nx, ny, pos_x, pos_y)
copy_glob_v = glob_v.copy()
write_local_to_global_vec(glob_v, loc_v, nx, ny, pos_x, pos_y)
print "Dummy diff (should be zero): ", np.linalg.norm(glob_v - copy_glob_v)

loc_res = np.dot(sqrt_term, loc_v)
write_local_to_global_vec(glob_v, loc_res, nx, ny, pos_x, pos_y)
fasit = np.dot(global_sqrt_term, copy_glob_v)

print "Result diff (should be zero): ", np.linalg.norm(glob_v - fasit)
print "Diff from multiplying the sqrt_term: ", np.linalg.norm(glob_v - copy_glob_v)
applying_sqrt_term_diff = glob_v - copy_glob_v

applying_sqrt_term_diff = applying_sqrt_term_diff.reshape((ny,nx))
fig = plt.figure(figsize=(4,4))
plt.imshow(applying_sqrt_term_diff, interpolation="None")
plt.title("applying_sqrt_term_diff ")
plt.colorbar()

glob_v = glob_v.reshape((ny,nx))
fig = plt.figure(figsize=(4,4))
plt.imshow(glob_v, interpolation="None")
plt.title("glob_v")
plt.colorbar()

copy_glob_v_eta_shape = copy_glob_v.reshape((ny,nx))
fig = plt.figure(figsize=(4,4))
plt.imshow(copy_glob_v_eta_shape, interpolation="None")
plt.title("copy_glob_v_eta_shape")
plt.colorbar()

print copy_glob_v_eta_shape[0,4], copy_glob_v[4]

# Testing the function that will be implemented on GPU
local_SVD_on_global_vector = apply_local_SVD_to_global_eta(sqrt_term, copy_glob_v_eta_shape, nx, ny, pos_x, pos_y) 
fig = plt.figure(figsize=(4,4))
plt.imshow(local_SVD_on_global_vector - glob_v, interpolation="None")
plt.title("local_SVD_on_global_vector - glob_v ")
plt.colorbar()

## Reading different files to see if the SVD terms are the same

The below code confirms that the local block for $U \Sigma^{1/2}$ is the same for different domain sizes and for different observation positions.


In [ ]:
sqrt_term_10_10_5_5 = np.load("svd_tests/local_sqrt_term_nx_10_ny_10_posx_5_posy_5.npy")
fig = plt.figure(figsize=(4,4))
plt.imshow(sqrt_term_10_10_5_5, interpolation="None")
plt.title("sqrt_term_10_10_5_5 ")
plt.colorbar()

sqrt_term_10_10_3_3 = np.load("svd_tests/local_sqrt_term_nx_10_ny_10_posx_3_posy_3.npy")
fig = plt.figure(figsize=(4,4))
plt.imshow(sqrt_term_10_10_5_5, interpolation="None")
plt.title("sqrt_term_10_10_5_5 ")
plt.colorbar()

sqrt_term_15_15_10_10 = np.load("svd_tests/local_sqrt_term_nx_15_ny_15_posx_10_posy_10.npy")
fig = plt.figure(figsize=(4,4))
plt.imshow(sqrt_term_15_15_10_10, interpolation="None")
plt.title("sqrt_term_15_15_10_10 ")
plt.colorbar()


sqrt_term_7_7_3_3 = np.load("svd_tests/local_sqrt_term_nx_7_ny_7_posx_3_posy_3.npy")
fig = plt.figure(figsize=(4,4))
plt.imshow(sqrt_term_7_7_3_3, interpolation="None")
plt.title("sqrt_term_7_7_3_3 ")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.imshow(sqrt_term_10_10_5_5 - sqrt_term_10_10_3_3 , interpolation="None")
plt.title("sqrt_term_10_10_5_5 - sqrt_term_10_10_3_3 ")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.imshow(sqrt_term_10_10_5_5 - sqrt_term_15_15_10_10 , interpolation="None")
plt.title("sqrt_term_10_10_5_5 - sqrt_term_15_15_10_10 ")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.imshow(sqrt_term_7_7_3_3 - sqrt_term_15_15_10_10 , interpolation="None")
plt.title("sqrt_term_7_7_3_3 - sqrt_term_15_15_10_10 ")
plt.colorbar()

# Creating matrix $S = (HQH^T + R)^{-1}$
$$S = (H U_{GB} \tilde{Q}^{1/2} \tilde{Q}^{1/2} U_{GB}^T + R)^{-1}$$

In [ ]:
print (nx, ny, ensemble.dx, ensemble.dy, pos_x, pos_y, q0, L)

H = createMatrixH(nx, ny, pos_x, pos_y)
Qtilde = createCutoffMatrixQ(nx, ny, dx=ensemble.dx, dy=ensemble.dy, L=0.75*dx, q0=q0)
U_GB = createUGBmatrix(nx, ny, g=ensemble.g, H=np.max(ensemble.base_H), f=ensemble.f, dx=ensemble.dx, dy=ensemble.dy)

print "g*H/(f*2*dx)", ensemble.g*np.max(ensemble.base_H)/(ensemble.f*2*ensemble.dx)

fig = plt.figure(figsize=(20,2))
plt.imshow(H, interpolation="None", cmap="cool")
plt.title("H")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.imshow(Qtilde, interpolation="None")
plt.title("Qtilde")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.imshow(U_GB, interpolation="None")
plt.title("U_GB")
plt.colorbar()
print "U_GB.shape: ", U_GB.shape

Q_sqrt = np.dot(U_GB, Qtilde)
fig = plt.figure(figsize=(4,4))
plt.imshow(Q_sqrt, interpolation="None")
plt.title("Q_sqrt")
plt.colorbar()

Q = np.dot(Q_sqrt, Q_sqrt.transpose())
fig = plt.figure(figsize=(4,4))
plt.imshow(Q, interpolation="None")
plt.title("Q")
plt.colorbar()

HQHT = np.dot(H, np.dot(Q, H.transpose()))
fig = plt.figure(figsize=(4,4))
plt.imshow(HQHT, interpolation="None")
plt.title("HQHT")
plt.colorbar()
print "HQHT"
print HQHT

print "S from matrix multiplications: "
print np.linalg.inv(HQHT + ensemble.observation_cov)

debug=False
S = createS(ensemble, 10.0)
print "S from createS function: "
print S

print "np.max(ensemble.base_H)", np.max(ensemble.base_H)